In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from urllib.parse import urljoin
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import mlflow
import matplotlib.pyplot as plt
import seaborn as sns
import socket
from urllib.parse import urlparse

def verify_local_server(uri):
    """Verifica que el URI apunte a un servidor local accesible"""
    try:
        parsed = urlparse(uri)
        if parsed.hostname not in ['localhost', '127.0.0.1']:
            raise ValueError(f"El host {parsed.hostname} no es una dirección local")
        
        # Verifica si el puerto está disponible
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.settimeout(2)
            if s.connect_ex((parsed.hostname, parsed.port)) == 0:
                return True
            else:
                raise ConnectionError(f"No se puede conectar al servidor MLflow en {uri}")
    except Exception as e:
        raise ConnectionError(f"Error verificando servidor local: {str(e)}")

# ===== Configuración Inicial con Verificaciones =====
try:
    # 1. Configuración NLTK (con verificación de recursos)
    nltk_data_path = "nltk_data"
    nltk.data.path.append(nltk_data_path)
    
    try:
        nltk.data.find('tokenizers/punkt')
        nltk.data.find('corpora/stopwords')
    except LookupError:
        print("Descargando recursos NLTK... (esto puede tomar 1-2 minutos)")
        nltk.download('punkt', download_dir=nltk_data_path)
        nltk.download('stopwords', download_dir=nltk_data_path)
        print("✓ Recursos NLTK instalados")
    
    analyzer = SentimentIntensityAnalyzer()
    
    # 2. Configuración MLflow con verificación explícita
    mlflow_uri = "http://127.0.0.1:5000"
    print(f"\nVerificando conexión con MLflow en {mlflow_uri}...")
    
    # verify_local_server(mlflow_uri) # Comentado si causa problemas y el servidor está corriendo
    mlflow.set_tracking_uri(mlflow_uri)
    
    # Verificación adicional
    if mlflow.get_tracking_uri() != mlflow_uri:
        raise ValueError(f"El URI de tracking no se estableció correctamente. Actual: {mlflow.get_tracking_uri()}")
    
    print("✓ Conexión con MLflow verificada")
    
    # Crear/verificar experimento
    experiment_name = "Books_Sentiment_Analysis"
    experiment = mlflow.get_experiment_by_name(experiment_name)
    if not experiment:
        print(f"Creando nuevo experimento: {experiment_name}")
        mlflow.create_experiment(experiment_name)
    mlflow.set_experiment(experiment_name)
    print(f"✓ Experiment configurado: {experiment_name}\n")
    
except Exception as e:
    print(f"\n× Error en configuración inicial: {str(e)}")
    print("Soluciones posibles:")
    print("1. Asegúrate de tener el servidor MLflow ejecutándose (ejecuta 'mlflow ui' en otra terminal)")
    print("2. Verifica que el puerto 5000 no esté bloqueado por tu firewall")
    print("3. Revisa tu conexión a internet para descargar recursos NLTK")
    # exit(1) # Considera no salir si es un entorno interactivo

# ===== 1. Web Scraping (Versión Corregida y Mejorada) =====
def scrape_books(initial_url):
    books = []
    current_url = initial_url
    page_count = 0
    max_pages = 50 # El sitio books.toscrape.com tiene 50 páginas de catálogo

    print(f"Iniciando scraping desde: {initial_url}")

    while current_url and page_count < max_pages:
        page_count += 1
        print(f"Procesando página {page_count}: {current_url}")
        
        try:
            response = requests.get(current_url, timeout=10)
            response.raise_for_status() # Lanza una excepción para errores HTTP (4xx o 5xx)
            
            # Usar response.url como base para urljoin, ya que es la URL final después de posibles redirecciones.
            actual_page_url = response.url 
            soup = BeautifulSoup(response.text, 'html.parser')
            
            book_articles = soup.find_all('article', class_='product_pod')
            if not book_articles:
                print(f"No se encontraron libros en {actual_page_url}. Deteniendo el scraping de esta rama.")
                break

            for book_tag in book_articles:
                title = book_tag.h3.a['title']
                
                relative_book_href = book_tag.h3.a['href']
                detail_url = urljoin(actual_page_url, relative_book_href)
                
                desc = "No description available" 
                try:
                    detail_response = requests.get(detail_url, timeout=5)
                    detail_response.raise_for_status() 
                    detail_soup = BeautifulSoup(detail_response.text, 'html.parser')
                    
                    desc_tag = detail_soup.find('div', id='product_description')
                    if desc_tag and desc_tag.find_next_sibling('p'):
                        desc = desc_tag.find_next_sibling('p').text.strip()
                    elif detail_soup.find('meta', attrs={'name': 'description'}):
                        meta_desc = detail_soup.find('meta', attrs={'name': 'description'})
                        if meta_desc and meta_desc.has_attr('content'):
                            desc = meta_desc['content'].strip()
                            
                except requests.exceptions.HTTPError as http_err_detail:
                    print(f"Error HTTP obteniendo descripción para '{title}' de {detail_url}: {http_err_detail}")
                except requests.exceptions.RequestException as req_err_detail:
                    print(f"Error de Red obteniendo descripción para '{title}' de {detail_url}: {req_err_detail}")
                except Exception as e_detail:
                    print(f"Error genérico obteniendo descripción para '{title}' ({detail_url}): {e_detail}")
                
                books.append({'title': title, 'description': desc})
                time.sleep(0.1) 
            
            next_btn_tag = soup.find('li', class_='next')
            if next_btn_tag and next_btn_tag.a and next_btn_tag.a.has_attr('href'):
                relative_next_href = next_btn_tag.a['href']
                current_url = urljoin(actual_page_url, relative_next_href)
            else:
                print("No hay más páginas o no se encontró el botón 'siguiente'.")
                current_url = None 
            
            time.sleep(1) 
            
        except requests.exceptions.HTTPError as http_err_main:
            print(f"Error HTTP al acceder a {current_url if current_url else 'URL desconocida'}: {http_err_main}")
            if http_err_main.response.status_code == 404:
                 print(f"Recibido 404 para {current_url if current_url else 'URL previa'}, el scraper se detendrá para esta ruta.")
            current_url = None # Detener si hay un error 404 o similar en la página principal
            break 
        except requests.exceptions.RequestException as req_err_main:
            print(f"Error de Red al acceder a {current_url if current_url else 'URL desconocida'}: {req_err_main}")
            current_url = None
            break 
        except Exception as e_main:
            print(f"Error inesperado durante el scraping de {current_url if current_url else 'URL desconocida'}: {e_main}")
            current_url = None
            break 
            
    return pd.DataFrame(books)

# ===== 2. Sentiment Analysis =====
def analyze_sentiment(df):
    # Verifica si la columna 'description' existe
    if 'description' not in df.columns:
        print("Advertencia: La columna 'description' no se encuentra en el DataFrame. El análisis de sentimiento podría no funcionar como se espera.")
        df['sentiment'] = 0.0 # o algún valor por defecto
        df['processed'] = ""
        return df

    def preprocess(text):
        tokens = word_tokenize(str(text).lower()) # Convertir a string para evitar errores con None o float
        stops = set(stopwords.words('english'))
        return ' '.join([w for w in tokens if w.isalpha() and w not in stops])
    
    df['processed'] = df['description'].apply(preprocess)
    df['sentiment'] = df['processed'].apply(lambda x: analyzer.polarity_scores(x)['compound'])
    return df

# ===== 3. MLflow Tracking Completo =====
def log_to_mlflow(df):
    # Iniciar experimento
    # mlflow.set_experiment("Books_Sentiment_Analysis") # Ya configurado globalmente

    # Verificar si hay datos para registrar
    if df.empty or 'sentiment' not in df.columns:
        print("No hay datos analizados para registrar en MLflow o falta la columna 'sentiment'.")
        return

    with mlflow.start_run():
        print("\n=== Registrando en MLflow ===")
        
        # 1. Parámetros (inputs)
        mlflow.log_param("source_url", "https://books.toscrape.com/")
        mlflow.log_param("num_books_processed", len(df))
        
        # 2. Métricas (resultados numéricos)
        avg_sentiment = df['sentiment'].mean()
        mlflow.log_metric("avg_sentiment", avg_sentiment)
        print(f"Sentimiento promedio: {avg_sentiment:.2f}")
        
        # 3. Artefactos (archivos)
        # Guardar datos
        try:
            df.to_csv("books_with_sentiment.csv", index=False)
            mlflow.log_artifact("books_with_sentiment.csv")
        except Exception as e:
            print(f"Error guardando o registrando 'books_with_sentiment.csv': {e}")

        # Gráfica mejorada
        try:
            plt.figure(figsize=(10, 6))
            sns.histplot(df['sentiment'], bins=20, kde=True)
            plt.title('Distribución de Sentimiento en Descripciones de Libros')
            plt.xlabel('Puntaje de Sentimiento')
            plt.ylabel('Número de Libros')
            plt.savefig("sentiment_distribution.png")
            mlflow.log_artifact("sentiment_distribution.png")
            plt.close() # Cerrar la figura para liberar memoria
        except Exception as e:
            print(f"Error generando o registrando 'sentiment_distribution.png': {e}")

        # 4. Etiquetas
        mlflow.set_tag("analysis_type", "sentiment_analysis")
        mlflow.set_tag("status", "completed" if not df.empty else "partial_data")
        
        print("Datos registrados (o intento de registro) en MLflow.")
        print(f"Run ID: {mlflow.active_run().info.run_id}")
        print("Para ver los resultados ejecuta: mlflow ui")

# ===== Ejecución Principal =====
if __name__ == "__main__":
    print("=== Iniciando Pipeline ===")
    
    # 1. Scraping
    print("\nExtrayendo datos de libros...")
    # Se usa la URL base, el scraper maneja la navegación a partir de /catalogue/page-1.html
    books_df = scrape_books("https://books.toscrape.com/") 
    print(f"Libros obtenidos: {len(books_df)}")
    
    if not books_df.empty:
        # 2. Análisis de sentimiento
        print("\nAnalizando sentimiento...")
        analyzed_df = analyze_sentiment(books_df.copy()) # Usar .copy() para evitar SettingWithCopyWarning
        
        # Mostrar algunos resultados
        print("\nMuestra de resultados:")
        if 'title' in analyzed_df.columns and 'sentiment' in analyzed_df.columns:
            print(analyzed_df[['title', 'sentiment']].head())
        else:
            print("No se pudieron mostrar los resultados ya que faltan las columnas 'title' o 'sentiment'.")

        # 3. MLflow
        try:
            log_to_mlflow(analyzed_df)
        except Exception as e_mlflow:
            print(f"Error durante el registro en MLflow: {e_mlflow}")
            print("Asegúrate de que el servidor MLflow esté activo en http://127.0.0.1:5000")
    else:
        print("No se obtuvieron libros, se omiten los pasos de análisis y registro en MLflow.")
    
    print("\n=== Pipeline completado ===")


Verificando conexión con MLflow en http://127.0.0.1:5000...
✓ Conexión con MLflow verificada
✓ Experiment configurado: Books_Sentiment_Analysis

=== Iniciando Pipeline ===

Extrayendo datos de libros...
Iniciando scraping desde: https://books.toscrape.com/
Procesando página 1: https://books.toscrape.com/
Procesando página 2: https://books.toscrape.com/catalogue/page-2.html
Procesando página 3: https://books.toscrape.com/catalogue/page-3.html
Procesando página 4: https://books.toscrape.com/catalogue/page-4.html
Procesando página 5: https://books.toscrape.com/catalogue/page-5.html
Procesando página 6: https://books.toscrape.com/catalogue/page-6.html
Procesando página 7: https://books.toscrape.com/catalogue/page-7.html
Procesando página 8: https://books.toscrape.com/catalogue/page-8.html
Procesando página 9: https://books.toscrape.com/catalogue/page-9.html
Procesando página 10: https://books.toscrape.com/catalogue/page-10.html
Procesando página 11: https://books.toscrape.com/catalogue/pa